In [3]:
import numpy as np
from itertools import combinations
from collections import defaultdict
import random
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
import time
from joblib import dump, load
import networkx as nx

In [4]:
edge_ind_count = defaultdict(int)
all_nodes = set()
with open("./Data/train.txt") as f:
    total_line = 0
    for line in f:
        total_line+=1
        nodes = list(map(int, line.strip('\n').split(' ')))
        for node in nodes:
            all_nodes.add(node)
        for comb in combinations(nodes,2):
            sorted_comb = tuple(sorted(comb))
            edge_ind_count[sorted_comb] += 1

all_nodes = list(all_nodes)

In [5]:
pos_samples = list(edge_ind_count.keys())

neg_pos_ratio = 1


neg_samples = []
for i in range(len(pos_samples)):
    while True:
        node1 = random.choice(all_nodes)
        node2 = random.choice(all_nodes)
        edge = tuple(sorted([node1, node2]))
        if edge not in pos_samples and node1 != node2:
            neg_samples.append(edge)
            break

In [6]:
print("length of positive samples: {}".format(len(pos_samples)))
print("length of negative samples: {}".format(len(neg_samples)))

length of positive samples: 16036
length of negative samples: 16036


In [7]:
# save
np.save("data/pos_samples", np.array(pos_samples))
np.save("data/neg_samples", np.array(neg_samples))

In [6]:
train_set = []
for edge in pos_samples:
    train_set.append([edge[0], edge[1], 1])
for edge in neg_samples:
    train_set.append([edge[0], edge[1], 0])

In [78]:
node_emb = {}
with open("embedding/emb64_p0.5q2.txt", "r") as f:
    next(f)
    for line in f:
        nums = line.strip().split(" ")
        node_id = int(nums[0])
        emb = list(map(float, nums[1:]))
        node_emb[node_id] = emb

In [79]:
emb_train_set = []
emb_train_X = []
emb_train_Y = []


for n1, n2, label in train_set:
    features = node_emb[n1] + node_emb[n2]
    emb_train_set.append(features + [label])
    emb_train_X.append(features)
    emb_train_Y.append(label)

In [80]:
import pandas as pd
test_set = pd.read_csv("Data/test-public.csv")

In [81]:
emb_test_X = []
missing_indices = []

for i in range(len(test_set)):
    idd, source, sink = test_set.iloc[i]
    if source not in node_emb or sink not in node_emb:
        emb_test_X.append([0 for _ in range(128)])
        missing_indices.append(idd)
        continue
    emb_source = node_emb[source]
    emb_sink = node_emb[sink]
    features = emb_source + emb_sink
    emb_test_X.append(features)